In [ ]:
# Logging und Decorators
import logging as log

# Tensorflow Module
import tensorflow as tf
import tensorflow_text as tf_text
from tensorflow.keras import layers

# Visualisierung und Eingabe
import ipywidgets as widgets
from ipywidgets import interact_manual, interactive, interact, VBox, HTML
from IPython.display import display, clear_output

# Backend Module
from interactive_inference_backend import ModelLoader, StoryTokenizer, WordComplete, VisualWrapper, positional_encoding
from interactive_inference_backend import reserved_tokens, vocab_path

In [ ]:
model = ModelLoader(StoryTokenizer(reserved_tokens, vocab_path),
                            d_model=512,
                            n_stacks=2,
                            h_att=4,
                            load_model=True,
                            model_load_path="model_N2_h4_d512_t20230831-134344")

# Interaktive Erklärung der Transformerarchitektur

## Inhaltsverzeichnis
- [Einleitung](#einleitung)
    - [Kurzübersicht](#kurzübersicht-transformer)
    - [Ziel des Artikels](#ziel-dieses-interaktiven-artikels)
    - [Architekturübersicht](#architekturübersicht)
    - [Encoder-Decoder Architektur](#encoder-decoder)
    - [Architekturblöcke](#architekturblöcke)
- [Input](#input)
    - [Tokenisation](#tokenisation)
    - [Byte-Pair Encoding](#byte-pair-encoding)
    - [Embedding](#embedding)
    - [Positional Encoding](#positional-encoding)
- [Trainingsmethoden](#trainingsmethoden)
    - [Dropout](#dropout)
    - [Normalization](#normalization)
    - [Residual Connection](#residual-connection)
- [Layers](#layers)
    - [Attention](#attention)
        - [Attention-Function](#attention-function)
        - [Multi-Headed Attention](#multi-headed-attention)
    - [Masking](#masking)
        - [Padding Masking](#padding-masking)
        - [Subsequent Masking](#subsequent-masking)
    - [Attention-Mechanismen](#verschiedene-attention-mechanismen)
        - [Self-Attention](#self-attention)
        - [Cross-Attention](#cross-attention)
        - [Masked Attention](#masked-attention)
    - [Skalierung mit sqr(d_k)](#skalierung-mit-sqrd_k)
- [Simulation](#vollständige-simulation)
- [Bibliographie](#bibliography)

## Einleitung
### Kurzübersicht Transformer

Transformer-Modelle sind eine von Vaswani et al. [1] vorgeschlagene Architektur für das Modellieren sequenzieller Daten, also Daten, die räumlich oder zeitlich geordnet sind, wie z.B. Zeitreihen, Text- und Videodaten. Im Gegensatz zu zuvor genutzten Architekturen wie Recurrent Neural Networks (RNN) [2, 3] oder Convolutional Neural Networks (CNN) [4] ermöglichen Transformer das parallele Verarbeiten sequentieller Daten. Dies ermöglicht, bei Verfügbarkeit entsprechend parallel arbeitender leistungsfähiger Hardware (wie insbesondere "GPUs" - Graphical Processing Units mit i.d.R. einigen tausend parallelen Recheneinheiten), oftmals eine signifikante Reduktion der Trainingszeiten [1, 6]. Typischerweise werden diesen Modellen Aufgaben gestellt, bei denen es darum geht, das nächste Element einer Sequenz vorherzusagen, wie beispielsweise den nächsten Datenpunkt einer Zeitreihe, das nächste Wort in einem Textabschnitt oder das nächste Bild in einem Video. Zur Veranschaulichung konzentrieren wie uns im Folgenden auf die Verarbeitung Textdaten.

Kernbestandteil von Transformermodellen bilden sog. "Attention"-Blöcke. Diese Blöcke steuern die Aufmerksamkeit bei der Datenverarbeitung, was bedeutet, dass nicht jedes Wort in einer Wortfolge dieselbe Bedeutung für die Prognose des nächsten Worts hat. Genauer gesagt verarbeitet das Modell nicht unbedingt ganze Wörter, sondern sog. "Token". Dabei handelt es sich in der Regel um häufig autretende Buchstabenkombinationen, die ganze Wörter, Wortteile oder zusammenhängende Wörter, in den zu lernenden Texten sein können. Attention-Blöcke sind mathematisch betrachtet trainierbare Matrizen, die die Eingabe (z.B. die Frage an einen Chatbot) auf die gewünschte Ausgabe (die Antwort auf die Frage) projizieren. Idealerweise projiziert das Transformer-Modell von der Eingabe genau die Information auf die gelernten Texte, sodass eine möglichst passende Antwort generiert wird. Dies lässt sich dem Schnelllesen von Texten vergleichen, bei dem die Attention-Blöcke relevante von nicht relevanten Textteilen filtern.  

Während des Trainings von Texten verwenden Transformer-Architekturen eine sog. "Maske", um einen natürlichen Lesefluss zu imitieren und gleichzeitig zu verhindern, dass das Modell Einblick in zukünftige Informationen erhält, die es prognostizieren soll. Mithilfe der Maske stützt sich die Prognose nur auf bereits bekannten Daten der Sequenz, das heißt, auf die zuvor gelesenen Wörter. Nehmen wir den Satz "Der Himmel ist klar und blau" als Beispiel: Während des Trainings werden die Wörter "Der Himmel ist" dem Modell präsentiert und es soll das Wort "klar" lernen. Die nachfolgenden Wörter wie "und blau" werden maskiert, da sie in dem natürlichen Lesefluss erst nach "klar" gelesen werden würden. Diese Technik etabliert eine Verbindung zwischen der aktuellen Position im Text und den bis zu diesem Punkt bekannten Informationen, ohne zukünftige Inhalte preiszugeben. Mit jeder Trainingsinstanz verschiebt sich die Maske schrittweise weiter über den Text, entsprechend dem Lesefluss. Während der Abfragephase, auch "Inference" genannt, wird dann zunächst das erste Token der Antwort generiert und im nächsten Schritt der Eingabe hinzugefügt, um das zweite Token zu generieren. Sobald die maximale Größe des sog. "Context-Windows" (=maximale Anzahl von Tokens in einer Eingabe) erreicht ist, wird das erste Token der Eingabe wieder entfernt. Dieses Context-Window und stellt quasi das Gedächtnis eines Transformer-Modells dar. 

Im Transformermodell werden sowohl Eingaben als auch Ausgaben intern als Vektoren repräsentiert. Die Umwandlung von Textworten in aussagekräftige Vektoren, die Funktionsweise weiterer Komponenten wie Attention-Blöcke und Feed Forward-Netze sowie die Rückführung von Vektoren zu Wörtern werden in den nachfolgenden Kapiteln beschrieben. Zudem werden zentrale Fachbegriffe und Konzepte wie „Positional Encoding“, „Embedding“ und „Normalisierung“ dargestellt. Darüber hinaus lassen sich alle Schritte in einem simulierbaren Transformermodell in diesem interaktiven Artikel schrittweise nachvollziehen.


### Ziel 

Das Ziel dieses Artikels besteht darin, die Transformerarchitektur aus [1] durch die Simulation ihrer Verarbeitungsschritte und Komponenten zu erläutern. Diese Simulation erlaubt es, interaktiv die Auswirkungen verschiedener Eingaben auf die Verarbeitungsschritte zu untersuchen und so ein schrittweises Verständnis der Gesamtverarbeitung zu entwickeln.   
Während sich wissenschaftliche Literatur wie der ursprüngliche Artikel [1] und darauf aufbauende wissenschaftliche Arbeiten [7, 8], Erklärartikel oder -videos [9, 10] sich oft auf einzelne Komponenten wie z.B. Attention-Blöcke konzentrieren, werden andere Elemente der Architektur wie z.B. "Dropout" [11], "Residual Connections" [12], "Byte-Pair Encoding" [13], Embedding oder des "Log-Softmax Algorithmus" [5] oft nicht oder nur rudimentär erklärt. Darüber hinaus ist insbesondere das Zusammenspiel der Elemente für die Transformer Architektur und ihr Verständnis wesentlich und soll in diesem Artikel ausführlich dargestellt und darüber hinaus interaktiv erfahrbar gemacht werden. 

Die Fachsprache aus dem Gebiet des Machine Learning ist in Englisch gehalten, um die Eindeutigkeit zu gewährleisten.

### Architekturübersicht

In <a href="#fig:fig1">Abbildung 1</a> findet sich eine Darstellung aller Architekturelemente der Transformerarchitektur mit ihren Verbindungen.
In der Darstellung werden vier Elemente farblich unterschieden.

<i>
<ul>
<li>Was ist mit den restlichen grafischen Unterscheidungen? z.B. Formen (z.B. von Lin Gen vs. Model Output) & Rand (gestrichelte Linien oder durchgezogene Linien) - notwendig? dann erklären sonst bitte entfernen
<li>Differenzierung mathematischer Operationen und Algorithmen z.B. Softmax vs. Tokenizer ggf. sinnvoll
<li>Was ist der Unterschied zwischen "Tokenized Sentence" und "Input Encoding"? -> keine Operation notwendig? Dann ein Block sonst eine Operation einfügen
<li> Abstraktionen in der Grafik in Relation zu Abbildung 2, Abbildung 2 in Abbildung 1 "auflösen" (wie besprochen)
<li> Query und Key werden gleich verarbeitet vom Input, Redundanzen entfernen
<li> Akronyme in der Abbildung auflösen / Unterschied zwischen "Pos Emb" und "Emb" unklar (s. Punkt 3) 
</ul> 
</i> 

In diesen Prozessen generierte Daten werden in Blau dargestellt. Trainingsparameter bzw. Gewichte des Modells werden in Gelb repräsentiert und diese repräsentieren das gelernte Wissen. Sog. "Hyperparameters" werden in Grau dargestellt. Sie unterscheiden sich von den Trainingsparametern, da sie bereits vor dem Training festgelegt werden und während des Trainings konstant bleiben (z.B. Anzahl der Layers und Neurons des Artificial Neural Networks). Ebenfalls in Grau werden Eingabe- und Ausgabedaten dargestellt.
Unsere Abbildung ist insofern komplett, als sie jeglichen Weg zeigen, den Daten durch das Modell nehmen können. Eine Darstellung, in der Form eines Encode-Decoder Network[14], also des Zusammenspiels eines Encoders, der die Eingabe enkodiert, mit einem Decoder, der die Ausgabe aus dieser Kodierung erstellt, findet sich in <a href="#fig:fig2">Abbildung 2</a>.

<figure id="fig:fig1" style="text-align: center; height: 700px;">
  <img src="./img/tf_arch_full.jpg" width="1000" height="1200" style="margin:auto;" alt="Eingabepipeline mit Tokenizer, Embedding und Positional Encoding."/>
  <figcaption>
  Abbildung 1: Transformerarchitektur <br>
  Das Bild veranschaulicht die Architektur eines Transformer-Modells. Besonderes Augenmerk wurde darauf gelegt, den Weg zu zeigen, den Informationen innerhalb des Transformer-Modells nehmen können, sowie die Funktionen darzustellen, die Veränderungen an diesen Informationen vornehmen. Prozesse und Methoden werden in Schwarz dargestellt. Die in diesen Prozessen generierte Daten werden in Blau dargestellt. Trainierbare Parameter werden in Gelb dargestellt. Hyperparameter, also festzulegende Eingabeparameter für das Modell werde in Grau dargestellt. Ebenfalls in Grau werden Eingabe- und Ausgabedaten dargestellt.
  <i>Anmerkungen s.o.</i>
  </figcaption>
</figure>
<br>


#### Encoder-Decoder

Eine Encoder-Decoder Architektur setzt sich aus zwei voneinander getrennten Neural Networks zusammen: dem Encoder und dem Decoder, die zusammen trainiert werden. Diese Architektur wurde von [14] vorgeschlagen und findet vor allem im Kontext von Übersetzungen. Der wesentliche Vorteil dieser Architektur besteht darin, dass sowohl der Encoder als auch der Decoder austauschbar sind. Theoretisch könnte man somit für jede Sprache einen Encoder und einen Decoder trainieren, um dann Übersetzungen zwischen beliebigen Sprachpaaren zu erstellen. Der Encoder komprimiert die Quellsprache in den Dimensionen des sogenannten "Latent Space" - eine bstrakte Repräsentation, die durch die latenten (versteckten) Dimensionen / Eigenschaften eines Modells gebildet wird. Der Decoder hingegen expandiert diese verdichteten Dimensionen zurück in die Zielsprache. Transformer für die Textvervollständigung können auch ausschließlich auf Decodern beruhen [16], dann übernehmen allein die Attention-Blöcke des Dekoders die Aufmerksamkeit. Bestimmte Aufgaben wie das Embedding der Eingaben sind allerdings weiterhin notwendig.

In unserem Beispiel wird der Encoder zum Enkodieren der Eingabe verwendet, um dann im Decoder die Eingabe als Informationsquelle für die Ausgabe zu verwenden. In <a href="#fig:fig2">Abbildung 2</a> sieht man wie Encoder und Decoder zusammenwirken. Der Decoder benutzt Self-Attention, um seinen Input zu verarbeiten, gefolgt von Cross-Attention und schließlich erneut Self-Attention, wohingegen der Encoder ausschließlich über Self-Attention funktioniert. Näheres siehe im Kapitel [Attention](#attention).

<figure id="fig:fig2" style="text-align: center; height: 700px;">
  <img src="./img/tf_encoder_decoder.jpg" style="margin:auto;" alt="Transformer als Encoder-Decoder."/>
  <figcaption>
  Abbildung 2: Transformer als Encoder-Decoder
  </figcaption>
</figure>
<br>

#### Architekturvarianten und verwirklichte Modelle


Transformer können auch als reine Encoder oder reine Decoder-Architektur verwendet werden. Encoder verdichten Informationen (z.B. zur Klassifikation, Sentiment-Analyse, oder Clustering), während Decoder generativ (z.B. zur Textfortsetzung oder Bildgenerierung) eingesetzt werden, um aus einer verdichteten Repräsentation wieder Informationen zu generieren. Wie beschrieben war die Encoder-Decoder Architektur vor allem für Natural Machine Translation gedacht. In der Umsetzung sieht man häufig reine Encoder oder Decoder Architekturen. Dies liegt an den Anforderungen ihrer jeweiligen Aufgaben, Architekturkomplexität und Trainingseffizienz. In der folgenden Tabelle [23] findet sich eine Auflistung der bekanntesten Modelle nach Architekturtyp.

| Encoder           | Encoder-Decoder | Decoder        |
|-------------------|-----------------|----------------|
| BERT, DistillBERT | T5              | GPT            |
| RoBERTa           | BART            | GPT-2          |
| XLM, XLM-R        | M2M-100         | GPT-3          |
| ALBERT            | BugBird         | GPT-4          |
| ELECTRA           |                 | GPT-Neo, GPT-J |
| DeBERTa           |                 |                | 

Hier können Sie nun einen Beispielsatz zuerst vom Encoder kodieren lassen, um ihn dann im nächsten Schritt vom Decoder dekodieren zu lassen und damit eine Ausgabe zu erhalten.


In [ ]:
tokenizer = StoryTokenizer(reserved_tokens, vocab_path)
transformer = model.model

input_widget_enc_dec = widgets.Text(
    value='Encoder-Decoder Test',
    description='Ihre Eingabe:',
    continuous_update=False,  # updates value only when you finish typing or hit "Enter"
    layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
)

button_widget_enc = widgets.Button(description='Enkodiere die Eingabe',
                               layout = widgets.Layout(width='auto'))
button_widget_dec = widgets.Button(description='Dekodiere die Input',
                               layout = widgets.Layout(width='auto'))

output_widget_enc = widgets.Output(layout = widgets.Layout(width='auto'))
output_widget_dec = widgets.Output(layout = widgets.Layout(width='auto'))


def encode():
    tensor_input = tf.convert_to_tensor(input_widget_enc_dec.value)            # Umwandelung des Textinputs in ein TensorFlow-Tensor 
    if len(tensor_input.shape) == 0:                                           # Überprüft, ob der Eingabetensor im korrekten Format ist
      tensor_input = tensor_input[tf.newaxis]                                  # Falls nicht, wird eine Dimension hinzufügt 
    
    tokenized_input = tokenizer.tokenize(tensor_input).to_tensor()             # Umwandlung des Textinputs in Tokens und anschließend in einen Tensor
    input_without_eos = tokenized_input[:, :-1]
    token_input = tokenizer.detokenize(input_without_eos)
    lookup = tokenizer.lookup(input_without_eos)
    lookup = [item.decode('utf-8') for sublist in lookup.numpy() for item in sublist]

    string_value = token_input.numpy()[0].decode('utf-8')
    
    context = transformer.encode(input_without_eos, None)                      # Kodierung des Inputsatzes von (Transformer-Modell)
    return context, string_value, lookup


def decode(): 
    tensor_input = tf.convert_to_tensor(input_widget_enc_dec.value)            # Umwandelung des Textinputs in ein TensorFlow-Tensor 
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)   # Erstellung eines leeren TensorArrays für die spätere Ausgabe
    if len(tensor_input.shape) == 0:                                           # wie bei der Encodierung
      tensor_input = tensor_input[tf.newaxis]                                  # wie bei der Encodierung

    tokenized_input = tokenizer.tokenize(tensor_input).to_tensor()             # wie bei der Encodierung
    input_without_eos = tokenized_input[:, :-1]
    
    token_input = tokenizer.detokenize(input_without_eos)
    string_value = token_input.numpy()[0].decode('utf-8')                      
    context = transformer.encode(input_without_eos, None)                     
    lookup = tokenizer.lookup(input_without_eos)
    lookup = [item.decode('utf-8') for sublist in lookup.numpy() for item in sublist]

                              
    for i, value in enumerate(tokenized_input[0][:-1]):                        # Schleife durch jedes Token des Satzes
      output_array = output_array.write(i, value)                              # Speichern des Tokens im Output array

    dec_input = output_array.concat()[tf.newaxis]                              # Output Array wird zu einem einzigen Tensor konkateniert 
                                                                               # und anschließend um eine zusätzliche Dimension erweitert

    dec_out = transformer.decode(context, None, dec_input, None)               # Decoder des Transformer-Modells wird verwendet, um den dec_input-Tensor 
                                                                               # unter Verwendung des zuvor berechneten Kontexts zu decodieren.

    return dec_out, string_value, lookup





def on_button_click_enc(b):
  with output_widget_enc:
    output_widget_enc.clear_output()  # clear the previous output
    context, tokens, lookup = encode()
    print('Wörter: ', tokens)
    print('Tokens: ', lookup)
    print('\n')
    #VisualWrapper.display_text('Beispieltext')
    VisualWrapper.color_bar(context) 


def on_button_click_dec(b):
  with output_widget_dec:
    output_widget_dec.clear_output()
    dec_out, tokens, lookup = decode()
    print('Wörter: ', tokens)
    print('Tokens: ', lookup)
    print('\n')
    #VisualWrapper.display_text('Beispieltext')
    VisualWrapper.color_bar(dec_out)



button_widget_enc.on_click(on_button_click_enc)
button_widget_dec.on_click(on_button_click_dec)

widgets.VBox([input_widget_enc_dec, widgets.HBox([widgets.VBox([button_widget_enc, output_widget_enc]), widgets.VBox([button_widget_dec, output_widget_dec])])])
#display(input_widget_enc_dec, button_widget_enc, output_widget_enc, button_widget_dec, output_widget_dec)


#print('tok_out', tokenized_input)
#print('enc_out', context)
#print("dec_out", dec_out)

#### Erklärung des Beispiels
Über der Grafik sind die verarbeiteten Tokens dargestellt. Die Positionierung der Tokens entlang der y-Achse der Grafik zeigt die Reihenfolge der Tokens im Beispielsatz an, wobei Leerzeichen nicht berücksichtigt werden. Das Model erkennt unterschiedliche Wörter mit Hilfe der "##" Zeichen. Durch diese Zeichen sieht das Modell, dass ein Token mit "##" Zeichenkette zu dem vorherigen Token gehört wie im Beispiel bei dem Wort "Endcoder", welches aus den Tokens 'e', '##n', '##co', '##der' besteht. Weitere Informationen in Abschnitt [Byte-Pair Encoding](#byte-pair-encoding).

Schaut man sich die Wörter vor der Tokenisierung an, sieht man das alle Wörter nun kleingeschrieben werden. Durch derartige Zusammenfassungen wird die Größe des Vokabulars reduziert. Linguistische und grammatische Informationen, die aufgrund der Zusammenführung verloren zu gehen scheinen, bleiben i.d.R. durch die Position und den Kontext des Wortes im Satz erhalten, wie zum Beispiel Substantivierungen. Dies führt dazu, dass das Modell stärker auf die Position des Wortes im Satz achtet. Der Tokenizer fügt zudem ein Start- und Endtoken hinzu. Das Endtoken wird im Modell nicht weiterverwendet, während das Starttoken die Position 0 in der weiteren Verarbeitung und Visualisierung einnimmt.

Die x-Achse repräsentiert die Tiefe der Token bzw. die Anzahl der Dimensionen des Vektors, der jedes Token darstellt. In diesem Fall beträgt die Tiefe 512, was bedeutet, dass jedes Token durch 512 verschiedene Werte charakterisiert wird. Diese Vektoren werden so gebildet, dass Tokens, die eine ähnliche Bedeutung haben oder aus einem Themenfeld stammen, ähnlichere Vektoren haben. Im Gegensatz dazu haben unähnliche Token eher unähnliche Vektoren. Aufgrund der hohen Dimensionalität der Vektoren können sich Ähnlichkeiten nur in den einzelnen Werten der Vektoren widerspiegeln. Dennoch ist ein gesamter Vektor im Vergleich zu anderen Vektoren aussagekräftig hinsichtlich seiner Gesamtorientierung. Zusätzlich ist in den Vektoren die Reihenfolge der Wörter im Satz kodiert, was als sogenannt [Positional Encoding](#positional-encoding) bezeichnet wird.  



#### Architekturblöcke

Prinzipiell lassen sich Transformer in mehrere Blöcke einteilen.

1. [Eingabeblock](#input)

Die Eingabepipeline verarbeitet die Eingabedaten in eine Form, die für die Matrixtransformation im Attention-Block genutzt werden kann. Diese unterscheidet sich für verschiedene Datentypen. In unserem Beispiel nutzen wir Textdaten, die wir durch Tokenization [17] und ein Embedding [18] in Tensoren verwandeln.

2. [Attention-Block](#attention)

Der Attention-Block verarbeitet Daten in Tensorform und liefert somit eine Abbildung von den Eingabetensoren auf die Ausgabetensoren, die jeweils von den Eingabe- und Ausgabeblöcken interpretiert wird.
Man kann verschiedene Modulvarianten innerhalb eines Attention-Blocks unterscheiden. Einzelne Attention-Module erhalten als Eingabedaten immer drei Tensoren. Diese werden Query, Key und Value genannt. Aus diesen berechnet ein Attention-Modul eine Ausgabe.
Typischerweise werden Attention-Module dadurch unterschieden aus welcher Quelle Query, Key und Value stammen.
Es gibt 

- Self-Attention bei der Query, Key und Value alle aus einer Quelle stammen und
- Source-Attention (Hier ist der Query aus einer anderen Quelle als Key und Value, z.B. wenn der Query die Ausgabe eines Encoders ist, während Target die Ausgabe eines Decoderblocks ist).

Desweiteren kann man nach Art des Maskings unterscheiden. Masking wird verwendet, um das Attention-Modul daran zu hindern aus einem Teil der Ausgabedaten zu lernen. In Transformern gibt es folgende Arten von Masking:

- Subsequent Masking, das den Decoder-Attention-Block daran hindert für die Vorhersage der Position i Daten aus den Positionen i+j zu nutzen, und
- Padding Masking, dass Fehler verhindert, die entstehen, da die Eingabe eines Transformers, unabhängig vom Inhalt, immer die selbe Anzahl an Tokens besitzt.

3. Ausgabeblock

Die Ausgabepipeline interpretiert die Ausgaben des Attention-Blocks, sodass sie in eine für menschlichen Gebrauch nützlichen Form vorliegen (typischerweise z.B. Textdaten, Bilddaten, etc.). Dafür wird in Transfomern eine Log-Softmax Funktion genutzt, die auf die Tensorausgabe des letzten Attention-Blockes angewandt wird.

In <a href="#fig:fig1">Abbildung 1</a> entspricht der Attention-Block den Prozessen innerhalb der grauen Umrandung, während die Eingabe- und Ausgabepipeline darunter bzw. darüber zu finden sind.


## Input

Der erste Teil eines Transformermodells besteht aus der Eingabepipeline. Diese verarbeitet die Eingabe, z.B. die Texteingabe eines Nutzers, und bereitet sie auf die Verarbeitung in den Attention-Modulen vor. Die Attention-Module arbeiten über eine Attention-Matrix, die aus der jeweiligen Eingabe eine Ausgabe berechnet. Wir müssen also aus einer Eingabe in Textform eine Vektorrepräsentation erzeugen, die alle notwendigen Informationen für das Modell enthalten, um mithilfe von Matrixmanipulationen nützliche Vorhersagen zu machen.

In <a href="#fig:fig3">Abbildung 3</a> sehen sie nochmal den Ausschnitt aus der obigen Grafik, der die Eingabepipeline darstellt. 
Wie zu erkennen ist, wird in der Eingabepipeline während des Training eines Transformermodells ausschließlich die Weights für das Embedding trainiert, alle anderen Funktionen sind rein deterministisch und bleiben damit vom Trainingsprozess unbeeinflusst.



<figure id="fig:fig3" style="text-align: center; height: 700px;">
  <img src="./img/tf_input_pipeline.jpg" style="height: 700px;" alt="Eingabepipeline mit Tokenizer, Embedding und Positional Encoding."/>
  <figcaption>Abbildung 2: Eingabepipeline eines Transformer-Netzwerks</figcaption>
</figure>
<br>


Prinzipiell besteht die Eingabepipeline aus drei Modulen:

1. Die [Tokenisation](#tokenisation), die den Text mithilfe eines Symbolalphabets in eine Zahlenkodierung umwandelt.
2. Das [Embedding](#embedding), das diese Kodierung mithilfe eines trainierbaren Algorithmus in eine Vektordarstellung umwandelt. Das Embedding lernt die komplexe Struktur eines Textes so darzustellen, dass sie informativ für die nachfolgenden Module ist. Wie genau das passiert ist dabei aufgrund der stochastischen Natur von Deep Learning Modellen nicht offen einsehbar.
3. Das [Positional Encoding](#positional-encoding) ein mit der Transformer-Architektur eingeführter Mechanismus. Im Gegensatz zu RNNs, die die Eingabedaten sequenziell präsentiert bekommen [19], enthalten bei Transformermodellen die Eingaben keine Information zur relativen Position der Tokens. Diese fehlenden Informationen werden in diesem Schritt manuell hinzugefügt.

### Tokenisation

Bei der Tokenisation wird der Satz in Textform z.B. "Das ist ein Testsatz." in einen Zahlencode verwandelt. Hierfür kommen verschieden Methoden in Frage. Einer der simpelsten Methoden ist es jedem Buchstaben eine eindeutige Zahl zuzuordnen. Da hier jeder Buchstabe einzeln kodiert werden muss, führt das allerdings zu einer sehr langen Kodierung. Obwohl das also eine mögliche Kodierung wäre, gibt es bessere Verfahren.

Die entscheidenden Faktoren für eine gute Kodierung sind
- Vollständigkeit der Kodierung, 
- Länge des kodierten Vektors und 
- Größe des dafür nötigen Vokabulars.

Die Kodierung mit einzelnen Buchstaben ist vollständig (man kann beliebige Zeichenkombinationen kodieren) und besitzt ein kurzes Vokabular (26 für alle Buchstaben plus Sonderzeichen), allerdings ist die Länge der kodierten Vektoren sehr groß.

Benutzt man ein Vokabular aus Wörtern, wird die Länge der Kodierung verkürzt, allerdings besteht die Gefahr der Unvollständigkeit. Um das nach Möglichkeit zu verhindern, benötigt man ein Vokabular, dass den Raum der möglichen Wörter vollständig abdeckt. Das Vokabular der Kodierung müsste also sehr groß sein.

Die Probleme mit obigen Methoden hat dazu geführt, dass sich gemischte Verfahren ergeben haben, die an einem möglichst großem Korpus trainiert werden. Es gibt einerseits Top-Down Verfahren, die von einem aus dem Korpus extrahierten Wort-Vokabular ausgehen und dann lernen unbekannte Worte in Teilworte zu zerlegen, die in das Vokabular mitaufgenommen werden. Andererseits gibt es Bottom-Up Verfahren, die von einem Buchstabenvokabular ausgehen und häufig wiederkehrende Kombinationen in dieses explizit aufnehmen.
Die Transformerarchitektur nach [1] verwendet ein solches Bottom-Up Verfahren namens Byte-Pair Encoding [13], dass sich als effizient erwiesen hat und ein relativ kompaktes Vokabular ermöglicht dabei aber die Länge der Tokenisation klein hält. 



### Byte-Pair Encoding

Das Byte-Pair Encoding Verfahren nutzt ein Vokabular mit einer festgelegten Länge. In unserer Implementation des Tokenizer nutzten wir ein Vokabular von der Länge 8000. Das Vokabular wird dabei folgendermaßen erstellt:

  1. Ein Text, der für die Erstellung des Vokabulars verwendet wird, wird in eine Sequenz von Symbole zerlegt. Wortenden werden mit einem zusätzlichen Symbol kodiert.
  2. Alle vorhandenen Symbole werden automatisch in das Vokabular aufgenommen.
  3. Nun wird das häufigste 2-Gramm, also zwei aufeinander folgende Symbole, gesucht, das im Text zu finden ist.
  4. Dieses wird ins Vokabular aufgenommen und im Text durch ein einzelnes Symbol ersetzt.
  5. Dieser Prozess wird nun wiederholt, bis die vorgegebene Länge des Vokabulars erreicht ist.

Dabei können sowohl ganze Wörter ins Vokabular aufgenommen werden, wenn sie denn oft genug auftauchen (bespielweise werden die Worte "a", "the", "and" bei englischen Texte sicherlich mitaufgenommen werden), aber auch einzelne Wortteile wie z.B. "en##", "##ment" oder "##ed" werden in diesem Vokabular sicherlich vorkommen, um seltene Kombinationen wie "enablement" in die Wortteile "en##", "able" und "##ment" zerlegen zu können oder grammatikalische Formen wie "wanted" zu bilden. Die Zeichenfolge "##" beschreibt dabei, dass hier kein Wortende stehen darf.

In unserem Testbeispiel ist zu sehen, wie Ihre Eingabe in Tokens getrennt und dann in eine Kodierung umgewandelt wird, je nachdem, welche Position das Token in unserem Vokabular hat.
Wie Sie sehen, enthält das Byte-Pair Encoding Vokabular auch ein [START]- und [END]-Token, sowie Elemente vom Typ 'abc##' oder '##abc'. Die Elemente mit Doppel-'#' stellen eine Sequenz am Anfang oder Ende eines Wortes dar.

In [ ]:
tokenizer = StoryTokenizer(reserved_tokens, vocab_path)

input_widget_tok = widgets.Text(
    value='Tokenizer test',
    description='Your input:',
    continuous_update=False,  # updates value only when you finish typing or hit "Enter"
    layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
)

button_widget_tok = widgets.Button(description='Run tokenizer on input',
                               layout = widgets.Layout(width='auto'))

output_widget_tok = widgets.Output(layout = widgets.Layout(width='auto'))


def tokenize(input_widget_tok):
    tokens = tokenizer.tokenize(input_widget_tok.value)                    # Erstellung der Tokens als Index für Vokabular
    lookup = tokenizer.lookup(tokens)                                      # Abrufen der Zeichenkette des Index im Vokabular                    
    
    return tokens, lookup
    
def on_button_click(b):
    with output_widget_tok:
        output_widget_tok.clear_output()                                                        
        tokens, lookup = tokenize(input_widget_tok)

        VisualWrapper.display_text('Tokens die aus der Eingabe mit Byte-Pair Encoding extrahiert werden:'.rjust(100) + ', '.join([token.decode('utf-8').rjust(10) for token in lookup.numpy()[0]])
                                   .replace(' ', '&nbsp;'))
        VisualWrapper.display_text('Ihre Positionsnummer im Alphabet des Byte-Pair Encoding Algorithmus:'.rjust(100) + ', '.join([str(token).rjust(10) for token in tokens.numpy()[0]])
                                   .replace(' ', '&nbsp;'))

button_widget_tok.on_click(on_button_click)

display(input_widget_tok, button_widget_tok, output_widget_tok)

### Embedding

Die Sequenzen die durch den Tokenizer entsteht variiert in ihrer Länge. Einen Einfluss darauf hat die Länge der Eingabe, aber auch durch die Eigenschaften des Byte-Pair-Encoding. Für die Parallelisierung des Trainingsprozesses ist es für Transformer aber notwendig eine gleichbleibende Eingabelänge beizubehalten.

Dies wird dadurch gelöst, dass Padding Tokens zum Einsatz kommen, also Tokens, die keine Information codieren, sondern der Eingabe beigefügt werden, um die erforderliche Länge zu erreichen.

Außerdem wird die Ausgabe des Tokenizers mithilfe einer trainierbaren Matrix in das notwendige Vektorformat gebracht

Das Embedding sorgt dafür, dass die Ausgabe des Tokenizer in einen Vektor der Modellgröße $d_{model}$ kodiert wird. Das heißt jedes Token, das zuvor durch eine Zahl kodiert wurde wird nun durch einen Vektor der Länge $d_{model}$ kodiert. 
Das Embedding  ist Teil der vom Modell gelernten Parameter (siehe <a href="#fig:embedding">Abbildung 3</a>) und somit nicht deterministisch gegeben. Welche Informationen aus der Ausgabe des Tokenizers wo gespeichert wird ist also nicht nachvollziehbar.
Vorstellen kann man sich aber, dass in jedem der $d_{model}$ Parametern einige der Informationen gespeichert werden, die für das jeweilige Token wichtig sind. Beispielsweise die Bedeutung des Tokens, seine grammatikalische Form, in welchem Kontext es benutzt wurde, steht es am Anfang eines Satzes oder am Ende, etc.

<figure id="fig:embedding" style="text-align: center; height: 300px;">
  <img src="./img/tf_embedding.jpg" style="height: 300px;" alt="Eingabepipeline mit Tokenizer, Embedding und Positional Encoding."/>
  <figcaption>Abbildung 3: Gewichte der Eingabepipeline</figcaption>
</figure>
<br>

Wie ein solches Embedding aussieht und wie es sich verändert, wenn man beispielsweise neue Teile an den Satz anfügt können Sie in der nachfolgenden Simulation ausprobieren. Der Eingabetext wird erst vom Tokenizer in Tokens umgewandelt und dann durch das Embedding in einen Tensor.

An jeder Position (vertikal dargestellt) ist dann das Embedding des Tokens an dieser Stelle zu sehen (horizontal dargestellt). Die farbliche Kodierung stellt dabei das Zahlenspektrum dar, indem sich die Vektoreinträge bewegen.

In [ ]:
class EmbeddingExample():

    def __init__(self) -> None:
        self.tokenizer = StoryTokenizer(reserved_tokens, vocab_path)

        self.input_widget = widgets.Text(
            value = 'Einbettung Test',
            description = 'Ihre Eingabe:',
            continuous_update=False,  # updates value only when you finish typing or hit "Enter"
            layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
        )

        self.button_widget = widgets.Button(description='Einbettung erstellen',
                                    layout = widgets.Layout(width='auto'))

        self.output_widget = widgets.Output(layout = widgets.Layout(width='auto'))
        self.old_context = None

    def create_tokenized_embeddings(self):
        tokens = self.tokenizer.tokenize(self.input_widget.value)                                 # Tokenisierung der Eingabe
        tokens_all = tokens[tf.newaxis, :, :]                                                     # Hinzufügen einer weiteren Dimension
        input_without_eos = tokens[tf.newaxis, :, :-1]                                            # Auswahl der Tokens bis zum [END] Token
        token_input = self.tokenizer.detokenize(tokens_all)                                       # Nur zur Ausgabe Zwecken
        string_value = token_input.numpy()[0][0].decode('utf-8')                                  # Nur zur Ausgabe Zwecken
        lookup = tokenizer.lookup(input_without_eos)                                              # Nur zur Ausgabe Zwecken
        lookup = [item.decode('utf-8') for sublist in lookup.numpy()[0] for item in sublist]      # Nur zur Ausgabe Zwecken
        print("Wörter: ", string_value)
        print("Tokens: ", lookup)
        context = model.model.enc_embed(input_without_eos)                                        # Erstellung des Kontext Embedding 
        VisualWrapper.display_text('So sieht die Einbettung der Eingabe aus.')
        VisualWrapper.color_bar(context.to_tensor())
        if self.old_context is not None:
             padded_context, padded_old_context = self.pad_tensors(context, self.old_context)     # Erstellung des Padding Vektors der Eingaben
             VisualWrapper.display_text('So unterscheiden sich die alte und die neue Einbettung voneinander.')
             context_diff = padded_context - padded_old_context                                   # Berechnung der Unterschiede beider Vektoren
             VisualWrapper.color_bar(context_diff)

        self.old_context = context

    
    def on_button_click(self, b):
        with self.output_widget:
            self.output_widget.clear_output()  # clear the previous output
            VisualWrapper.reset_visualiser()
            self.create_tokenized_embeddings()
    
    def pad_tensors(self, ragged_tensor1, ragged_tensor2):
        """Funktion um die Tensoren der Eingabe auf die gleiche Länge zu transformieren"""
        tensor1 = ragged_tensor1.to_tensor()                                                     # Umwandlung in normalen Tensor
        tensor2 = ragged_tensor2.to_tensor()                                                     # Umwandlung in normalen Tensor

        shape1 = tf.shape(tensor1)
        shape2 = tf.shape(tensor2)

        target_shape = []

        for i in range(shape1.shape[0]):                                                         # Iterieren über die Dimensionen der Tensoren
            target_shape.append(tf.maximum(shape1[i], shape2[i]))                                # Die maximale Größe der Dimension wird an die Zielform angehängt.

        target_shape = tf.stack(target_shape)                                                    # Umwandlung der Zielform in einen Tensor


        paddings1 = []
        paddings2 = []

        for i in range(shape1.shape[0]):                                                         # Iterieren über die Dimensionen der Tensoren
            paddings1.append([0, target_shape[i] - shape1[i]])                                   # Auffüllung der Tensor auf maximale Länge 
            paddings2.append([0, target_shape[i] - shape2[i]])                                   # Auffüllung der Tensor auf maximale Länge 

        paddings1 = tf.stack(paddings1)                                                          # Konvertieren der Paddings in Tensoren
        paddings2 = tf.stack(paddings2)                                                          # Konvertieren der Paddings in Tensoren

        tensor1_padded = tf.pad(tensor1, paddings1)                                              # Tensoren an die Zielform anpassen
        tensor2_padded = tf.pad(tensor2, paddings2)                                              # Tensoren an die Zielform anpassen

        return tensor1_padded, tensor2_padded

emb_ex = EmbeddingExample()

VisualWrapper.display_text('Hier kannst du einen Text einbetten lassen. Wenn du die Eingabe veränderst wird außerdem gezeigt, wie sich die Einbettung geändert hat.')

emb_ex.button_widget.on_click(emb_ex.on_button_click)
display(emb_ex.input_widget, emb_ex.button_widget, emb_ex.output_widget)


#### Erklärung des Beispiels
In diesen Beispiel wie ein Embedding grafisch aussieht. Das Embedding wird in der Funktion *create_tokenized_embeddings()* erstellt. Dazu wird zu erst der Eingabetext vom Tokenizer in Tokens unterteilt (Zeile 20). Die Tokens können Sie über der Grafik sehen. In Zeile 29 werden diese dann vom Transformer Modell in die Embeddings umgewandelt. 

Verwendet man zum Beispiel das Eingabebeispiel "Einbettung Test" und als zweites "Einbettung Test neu", sieht man im zweiten Bild wie die zusätzlichen Positionen dazu kommen. Die Visualisierung von "Einbettung Test" zeigt auf der y-Achse sieben Tokens. Bei dem Satz "Einbettung Test neu" kommen dann 3 Tokens dazu, diese sind "#n", "##e", "##u ". Ebenso kann man ein Muster bei dem Wertebereich erkennen. Der Bereich der Tiefe zwischen 0bis 256 bewegt sich hauptsächlich im Wertebereich -1 bis 2 und der Bereich von 257 bis 512 im Wertebereich 0 bis -3. Dies geht auf die Sinus-Funktion des Positional Encodings zurück, welche auf das Embedding angewendet wird. Diese hat für diese Bereiche stark unterschiedliche Werte. 

Die Werte in den ersten 256 Positionen können bestimmte Merkmale oder Eigenschaften der Wörter repräsentieren, während der Bereich der Positionen von 266 bis 512 andere Merkmale oder Eigenschaften widerspiegelt. Diese getrennte Darstellung ermöglicht dem Modell, komplexe Beziehungen und Muster in den Daten zu erfassen.




### Positional Encoding

Da im Embedding keine Informationen über die relative Position der verschiedenen Worte kodiert wird, muss diese manuell hinzugefügt werden. Hierfür benutzt die Transformerarchitektur für jede Position des Embedding (also jedes enkodierte Wort) eine Sinuskurve mit anderer Frequenz und Phase. [1] Der sich ergebende Wert wird dem Embedding an der jeweiligen Stelle hinzugefügt.

Dadurch lassen sich die verschiedenen Worte sehr gut voneinander trennen. Die Idee dahinter ist, dass sich:
1. die grobe Position eines Worte anhand der langfrequenten Sinuskurven bestimmen lässt, da sie sich über die gesamte Länge der Eingabe nur allmählich verändern und die Werte des Embedding durch diese insgesamt in eine bestimmte Richtung verschoben werden, z.B. die Worte im hinteren Teil der Eingabe größere Werte besitzen als die im vorderen Teil der Eingabe,
2. die genaue Position durch die hochfrequenten Sinuskurven bestimmen lässt, da diese sich schon für benachbarte Vektoren klar unterscheiden und somit klar wird, welches Wort an welcher Stelle im Embedding kodiert wurde.

$$
PE(pos, i) =
\begin{cases}
  \sin\left(\frac{pos}{10000^{2i/d_{\text{modell}}}}\right) & \text{falls } i \text{ gerade ist} \\
  \cos\left(\frac{pos}{10000^{2i/d_{\text{modell}}}}\right) & \text{falls } i \text{ ungerade ist}
\end{cases}
$$

- $pos$ die Position des Tokens in der Sequenz ist.
- $i$ der Index der Dimension in der Positionsverschlüsselung ist.
- $d_{model}$ die Dimensionalität des Modells ist.

In der unten stehenden Simulation ist zu sehen, wie das positional Encoding beipielhaft für ein 2048 x 512 langes und tiefes Embedding aussieht.

In [ ]:
@interact(length=(2,2048,1), depth=(2,512,1))
def print_pos_enc(length, depth):
    VisualWrapper.color_bar(positional_encoding(length, depth))

#### Erklärung des Beispiels
An diesem Beispiel sieht man den Effekt des Positional Encodings auf verschiedene lange bzw. tiefe Einbettungen. Wenn man mit den zwei Reglern spielt, sieht man, verschieden große Zooms auf den Effekt. Erhöht man den "length"-Regler sieht man die Auswirkungen, welche die Länge des Satzes betreffen. Die Werte die durch das Positional Encoding zu dem Vektor hinzugefügt werden sind dabei jedoch immer dieselben. Zum Bespiel wird an Stelle 100 immer derselbe Wert hinzugefügt, egal ob die maximale Länge des Satzes ("length") sich verändert. Dasselbe gilt auch für die Tiefe jedes einzelnen Vektors.


## Trainingsmethoden

### Dropout

Dropout [11] ist eine Methode, die während des Trainingsprozesses eines neuronalen Netzes genutzt wird, um zu verhindern, dass die gelernte Ausgabe eines Modells sich zu sehr auf einen einzelnen Prädikator stützt.
Dafür wird zwischen zwei Schritten desselben Modells, eine Dropout-Layer eingefügt. Diese setzt zufällig einige der vom ersten Modellteil generierten Ausgabe auf einen vordefinierten Wert (meistens -inf), um den nachfolgenden Schichten diese Information vorzuenthalten. Da diese Operation zufällig erfolgt, müssen die nachfolgenden Teile des Modells lernen ihre Ausgabe auch ohne diese Information zu erstellen. Somit lernt das Model seine Vorhersage auf eine möglichst breite Kombination an Merkmalen aufzubauen und man verhindert, dass Vorhersagen nur aufgrund eines einzigen Merkmals der vorherigen Ausgabe gemacht werden.

Ein gutes Beispiel ist das Ende eines Satzes vorherzusagen. In europäischen Sprachen wird ein Satz fast immer mit einem Punkt beendet, also ist es eine gute Strategie zu lernen, dass ein Satz durch einen Punkt beendet wird. Doch ein Modell, dass einen Punkt als einziges Merkmal eines Satzendes nutzt ist wenig robust. Wenn man an falscher Stelle einen Punkt setzt oder ihn an einem Satzende durch ein anderes Zeichen ersetzt werden die Vorhersagen des Models schlecht sein. Dabei gibt es auch andere Hinweise auf ein Satzende, z.B. das Vorkommen eines Verbs in der deutschen Sprache oder von Ort und Zeitangaben im Englischen.

Um dem Modell keine Informationen vorzuenthalten, wenn es tatsächlich eingesetzt wird, ist das Dropout immer nur während des Trainings aktiv und wird danach abgeschalten, sodass während der Inferenzphase keine Informationen gelöscht werden.

Wie sich eine Ausgabe eines Transformermodells unterscheiden kann, wenn man Dropout anwendet können Sie in der nachfolgenden Simulation sehen.

In der ersten Grafik sieht man welche Werte in einem uniformen Vektor vom Dropout verändert werden. In den beiden darauffolgenden Grafiken wird das Dropout auf den im Textfeld eingegebenen Beispieltext angewandt, nachdem er durch den Tokenizer und ein Embedding in Vektorform gebracht wurde. Die erste Grafik zeigt den vollständigen Vektor und die zweite Grafik den Vektor, der vom Dropout verändert wurde.

In [ ]:
tokenizer_drop = StoryTokenizer(reserved_tokens, vocab_path)

input_widget_drop = widgets.Text(value = 'Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test',
                                 description = 'Ihre Eingabe:',
                                 continuous_update=False,  # updates value only when you finish typing or hit "Enter"
                                 layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
                                 )

length_widget_drop = widgets.IntSlider(value=30,
                           min=2,
                           max=2048,
                           description='Länge des Tensors:',
                           continuous_update=False,  # updates value only when you finish typing or hit "Enter"
                           )
depth_widget_drop = widgets.IntSlider(value=512,
                          min=2,
                          max=512,
                          description='Tiefe des Tensors:',
                          continuous_update=False,  # updates value only when you finish typing or hit "Enter"
                          )
dropout_widget = widgets.FloatSlider(value=0.1,
                              min=0,
                              max=0.9,
                              step=0.1,
                              description='Dropoutrate:',
                              continuous_update=False,  # updates value only when you finish typing or hit "Enter"
                              )



def dropout_function(length, depth, dropout, input):

    # Erstellung der Dropout Layer
    dropout_layer = layers.Dropout(dropout)                                                         # Anwendung des Dropout auf die Layer
    one_tensor = tf.ones([length, depth])                                                           # Erstellung eines Arrays aus 1-en
    dropout_tensor = dropout_layer(one_tensor, training=True)                                       # Anwendung des Dropout auf die Layer

    # Erstellung der Kontext Layer
    tokens = tokenizer_drop.tokenize(input)                                                         # Tokenisierung des Inputs
    input_without_eos = tokens[tf.newaxis, :, 1:-1]                                                 # Auswahl der Tokens bis zum [END] Token
    context = model.model.enc_embed(input_without_eos)                                              # Erstellung des Embedding durch das Modell
    context_drop = dropout_layer(context, training=True)                                            # Anwendung des Dropout auf das Embedding

    return dropout_tensor, context_drop, context

def out(length, depth, dropout, input):
    VisualWrapper.reset_visualiser()                                                   
    dropout_tensor, context_drop, context = dropout_function(length, depth, dropout, input)
    VisualWrapper.color_bar(dropout_tensor)
                               
    VisualWrapper.color_bar(context.to_tensor())                     
    VisualWrapper.color_bar(context_drop.to_tensor())
    

output_widget_dropout = widgets.interactive_output(out,
                                                   {'length': length_widget_drop, 'depth': depth_widget_drop, 'dropout': dropout_widget, 'input': input_widget_drop}, 
                                                   )

display(length_widget_drop, depth_widget_drop, dropout_widget, input_widget_drop, output_widget_dropout)


#### Erklärung des Beispiels
In diesem Beispiel wird der Effekt von Dropout auf die Layer und das Embedding dargestellt. Dafür werden in der Funktion *dropout_function()* jeweils Dropout auf die Layer und auf das Embedding angewendet. Dafür wird in Zeile 36 und 42 jeweils die jeweilige Transformation auf den beiden Objekten angewendet. Damit wird also ein gewisser Teil, welche mit dem Parameter "Dropoutrate" bestimmt wird, der Werte Layer bzw. des Embeddings den weiteren Verarbeitungsschritten vorenthalten. Dieser Parameter ist ein prozentualer Wert, d.h. bei einem Wert von 0.2 werden 20% der Werte vorenthalten. 

Für das Beispiel können dieses Mal die Länge des Tensors (Eingabe) und die Tiefe des Tensors bestimmt werden. Ebenso kann die Dropoutrate verändert werden. In der ersten Grafik sieht man welche Werte in einem uniformen Vektor vom Dropout verändert werden. Das bedeutet, welche der Werte in der Tiefe des Tensors ausgeblendet werden. Der Wert ist entspricht dabei einer Prozentzahl bei 0.4 werden somit 40 Prozent der Werte ausgeblendet bzw. auf den festgesetzten Wert gesetzt.
In den beiden darauffolgenden Grafiken wird das Dropout auf den im Textfeld eingegebenen Beispieltext angewandt, nachdem er durch den Tokenizer und die Einbettung in Vektorform gebracht wurde. Die erste Grafik zeigt den vollständigen Vektor und die zweite Grafik den Vektor, der vom Dropout verändert wurde. Hier sieht man die ausgelassenen Positionen sehr gut. Mit dem Erhöhen der Dropout Rate, werden diese mehr. Außerdem kann man erkennen, dass sich, wenn man den Dropout erhöht, der Wertebereich ebenfalls ausweitet. Erklärbar ist dies dadurch, dass Merkmale, vor allem prägnante Merkmale, sich durch das weitere Training verstärken bzw. abschwächen. Die Dropoutrate kann jedoch nicht beliebig erhöht werden bei gleichzeitigem Erhalt der Güte der Klassifikation.
 


### Normalisierung

Normalization ist eine Technik, die von [20] eingeführt wurde. In Deep Neural Networks, die mit nicht-linearen Aktivierungsfunktionen wie der Sigmoid-Funktion

$$g(x) = 1/(1+exp(-x))$$

trainiert werden gilt, dass $g’(x) -> 0 für |x|-> inf$. 

Das führt dazu, dass diese Modelle in einen Bereich geraten können, in der der Gradient für das Stochastic Gradient Descent (SGD) minimal wird und man vom Vanishing Gradient Problem spricht.

In neuronalen Netzen ist hierbei das Problem, dass eine Layer $z = g(Wx + b)$ mit der Sigmoid-Funktion g versucht den Output des gesamten vorherigen Netzes x zu gewichten. Dabei hängen sowohl W als auch b von x ab. Da sich während des Trainings alle Layers des Netzes fortwährend aktualisieren, ändert sich auch der Input x fortwährend. 
Dieser Effekt wird von [20] Internal Covariate Shift genannt. 

Je tiefer das neuronale Netz, umso größer sind diese Veränderungen, da es mehr Schichten gibt, die sich verändern können. Die Tiefe einer neuronalen Netzes erhöht also die Wahrscheinlichkeit das ein Vanishing Gradient Problem auftritt und ein effektives Training frühzeitig aufhört.

Transformer wie sie in [1] beschrieben sind nutzen hierfür Layer Normalization, einen Normalisierungsalgorithmus den [21] entwickelt hat.


In [ ]:
VisualWrapper.reset_visualiser()
tokenizer_norm = StoryTokenizer(reserved_tokens, vocab_path)

input="Test"

tensor_input = tf.convert_to_tensor(input)
if len(tensor_input.shape) == 0:
    tensor_input = tensor_input[tf.newaxis]

tokenized_input = tokenizer_norm.tokenize(tensor_input).to_tensor()                             # Anwendung eines Tokenizers mit Normalisierung
input_without_eos = tokenized_input[:, :-1]
context = model.model.encode(input_without_eos, None)

VisualWrapper.visualize_data(id='layer')



### Residual Connection

Die Idee für das Nutzen von Residual Connections kommt von [22]. Die Autoren stellten fest, dass bei neuronalen Netzen sowohl die Genauigkeit während des Trainings als auch die Genauigkeit auf dem Testdatensatz mit zunehmender Tiefe schlechter wird.

Da durch Normalization bereits sichergestellt ist, dass das Vanishing Gradient Problem nicht auftritt, scheitert die Optimierung der neuronalen Netze aus anderen Gründen.
Einer der Gründe hierfür liegt vermutlich darin, dass die tieferen Schichten eines Modells zu Beginn des Trainings sehr viel stärker zur Ausgabe beitragen, als die vorhergehenden Schichten. Sie werden somit zuerst trainiert. Die weniger tiefen Schichten werden erst ausreichend trainiert, wenn in den tiefen Schichten keine Optimierung mehr möglich ist. 

Eine Lösung hierfür sind Residul Connections.
Sie ersetzen eine Schicht F(x) durch ihre Residual Connection

\
$$H(x) = F(x) + x.$$

<br>
In das Ergebnis von H(x) geht also sowohl der Output, als auch der Input von F direkt mit ein. Wendet man dieses Prinzip auf die Schichten tiefer neuronaler Netze an, sorgt das dafür, dass gleich zu Beginn, der Output der wenig tiefen Schichten relevant in den Output des gesamten Netzes einfließt, denn es gilt für das gesamte Netz N:

\
$$N(x) = H_n(H_n-1(x)) + H_n-1(x) = H_n(H_n-1(x)) + H_n-1(H_n-2(x)) + … + H_2(H_1(x)) + H_1(x)$$

<br>
Wie man in <a href="#fig:fig1">Abbildung 1</a> sehen kann, haben alle Attentionsmodule, sowie alle Feed-Forward Schichten in einem Transformermodell eine residuale Verbindung.

In der nachfolgenden Simulation können Sie sehen, wie sich die Ausgabe einer neuronalen Schicht verändert, wenn man ihr eine Residual Connection beifügt. Der Effekt auf den gesamten Trainingsprozess lässt sich dabei natürlich nur schwer darstellen.


## Layers

### Attention

Die Neuerung von Transformern im Vergleich zu vorangegangenen Lösungen für Neural Machine Translation (NMT) ist es, allein auf Attention als Mechanismus für das Verarbeiten von Sprache zu setzen. Attention wurde auch vorher schon von [15] zur Verbesserung von RNNs zur Übersetzung von Texten verwendet.


Die Einführung des Attention-Mechanismus in Transformer-Architekturen hat zu bedeutenden Verbesserungen in der Verarbeitung natürlicher Sprache geführt, insbesondere im Vergleich zu traditionellen rekurrenten neuronalen Netzwerken (RNNs). Einer der herausragenden Vorteile des Attention-Mechanismus ist seine Fähigkeit zur parallelen Verarbeitung von Daten. Im Gegensatz zu den sequenziellen Verarbeitungsgrenzen von RNNs ermöglicht diese Eigenschaft eine wesentlich effizientere Datenverarbeitung.

Ein weiterer entscheidender Fortschritt, den der Attention-Mechanismus mit sich bringt, ist sein flexibles Gedächtnis. Im Unterschied zu dem festen, oft begrenzten Gedächtnis der RNNs, erlaubt der dynamische und kontextabhängige Speicher des Attention-Mechanismus eine umfassendere und flexiblere Berücksichtigung von Informationen. Dies ist besonders nützlich für das Verständnis und die Verarbeitung komplexer Sprachstrukturen.

Besonders bemerkenswert ist auch, wie der Attention-Mechanismus die Handhabung von Langzeitabhängigkeiten verbessert. Durch die Fähigkeit, direkte Verbindungen zwischen weit auseinanderliegenden Elementen einer Sequenz herzustellen, können Transformer-Modelle effektiver mit Langzeitabhängigkeiten umgehen, was bei RNNs oft eine Herausforderung darstellt. Diese Fähigkeit verbessert das Verständnis und die Generierung von Sprache über längere Textabschnitte hinweg erheblich.

Schließlich ermöglicht der Attention-Mechanismus eine verbesserte Kontextverarbeitung. Die Fähigkeit, die Bedeutung von Wörtern und Phrasen im Kontext ihres Auftretens zu erfassen, führt zu einem präziseren und tieferen Verständnis der Sprache. Diese kontextuelle Bewusstheit, die über die Fähigkeiten traditioneller RNNs hinausgeht, ist entscheidend für anspruchsvolle sprachverarbeitende Aufgaben wie z.B. Übersetzung oder Zusammenfassungen.

Der Attention-Mechanismus, wie ihn [1] beschreiben, orientiert sich dabei an der Idee einer Suchanfrage des Ausgabetextes and sich selbst, bzw. den Eingabetext. Die Attention-Layer bekommt dabei zwei oder eigentlich drei Eingaben: 

1. den Query (Q), 
2. den Key (K),
3. den Value (V).

In der Praxis erhalten aber Key und Value in Transformern immer dieselbe Eingabe und häufig sind Query, Key und Value sogar alle identisch. Aus welcher Quelle Query, Key und Value kommen unterscheidet verschiedene Formen von Attention. So nennen wir Self-Attention denjenigen Fall indem Q=K=V gilt und Cross-Attention denjenigen Fall indem der Query aus der Ausgabe des Encoder besteht und Key und Value beide aus der Ausgabe eines Decoder-Blocks stammen.

Um zu erklären, wie Attention funktioniert, sollten wir aber zunächst davon ausgehen, dass Query-, Key- und Value-Eingabe verschieden sind. Ich schreibe bewusst von der Eingabe, da in jeder Attention-Layer zunächst Query-, Key- und Value-Eingabe Q', K', V' mit Hilfe von gewichteten Matrix W^Q, W^K und W^V in Query, Key und Value 

$$Q=Q′×W^Q$$
$$K=K′×W^K$$
$$V=V′×W^V$$ 

umgewandelt werden. Diese gewichteten Matrizen W^Q, W^K, W^V sind die trainierbaren Weights der Attention-Layer. Alle nachfolgenden Prozesse sind deterministischer Natur. Das bedeutet, dass diese Matrizen festlegen, welches Ergebnis die Attention-Layer liefert.
Eine grafisch aufbereitete Erklärung dessen, was hier beschrieben wird findet sich übrigens bei [9].

Attention unterscheidet sich deutlich von vorherigen Verfahren. Am ehsten ist es vergleichbar mit der oftmals bei der Programmierung verwendeten Datenstruktur "Dictionary". Ein Dictionary ist eine unsortierte Liste mit "key-value" Paaren. Hierbei lässt sich ein Wert, Objekt, Variable (Value) in der Liste speichern, welcher wiederum über den key abgefragt werden kann. Folgende Grafik zeigt die Gemeinsamkeiten und Unterschiede zu dem Attention-Mechanismus.    

<figure id="fig:fig_masking" style="text-align: center; height: 370px;">
  <img src="./img/tf_attention.png" alt="Abbildung 4: Vergleich von Attention mit Datenstruktur "Dictionary"."/>
  <figcaption>Abbildung 4: Vergleich von Attention mit einem klassischen Dictionary</figcaption>
</figure>
<br>

### Erklärung der Grafik


In beiden dargestellten Szenarien, "Dictionary" und "Attention", wird das Konzept von Key-Value-Paaren genutzt, wobei die Keys als Referenzindizes fungieren und die Values die eigentlichen zu verarbeitenden Informationen enthalten. Queries dienen in beiden Fällen dazu, relevante Daten aus diesen Paaren zu selektieren, und am Ende wird jeweils ein Output generiert, der aus den Informationen der Value-Komponenten resultiert.

Jedoch gibt es markante Unterschiede zwischen den beiden Ansätzen. Im Dictionary findet eine direkte Zuordnung statt, bei der ein Query-Element einem Key zugeordnet und das zugehörige Value direkt als Output übernommen wird. Bei "Attention" wird hingegen eine gewichtete Kombination der Values vorgenommen, die von der Relevanz der Keys, bestimmt durch die Queries, abhängt. Dies spiegelt sich auch in der Art der Beziehungen wider: Während im "Dictionary" eine eindeutige 1:1-Beziehung herrscht, besteht im "Attention"-Mechanismus eine 1:n-Beziehung, bei der ein Query mehrere Keys beeinflusst. Dementsprechend ist die Ausgabe im "Dictionary" statisch und hängt ausschließlich von der direkten Übereinstimmung ab, während sie im "Attention"-Modell dynamisch ist und durch die berechneten Gewichtungen eine nuanciertere Informationszusammenstellung ermöglicht.

#### Attention Function

Die Attentionsfunktion bekommt die Eingaben Q, K, V aus der vorher beschriebenen Gewichtung und lautet:
<br>
<br>
$$Attention(Q, K, V)=softmax(QK^T/sqr(d_k))V$$
<br>
Es wird also zuerst das Kreuzprodukt aus Q und K gebildet. Dieses Produkt wird mit sqr(*d_k*) skaliert, weshalb ist in folgendem [Kaptiel](#skalierung-mit-sqrd_k) nachzulesen, und auf dieses Ergebnis wird dann die Softmax-Funktion sigma(x) angewandt.
Diese Funktion lässt sich am besten positionsweise beschreiben
<br>
<br>
$$sigma(x)_i = exp(x_i)/sum_j=1^n(exp(x_j)) für i=1,...,n.$$
<br>
Nennen wir also
<br>
$$softmax(QK^T/sqr(d_k)) = Score(Q,K),$$
<br>
dann ist 
<br>
$$Attention(Q, K, V) = Score(Q,K)*V $$
<br>
eine Funktion, die $V$ mit einem Vektor multipliziert, wobei für den Vektor gilt $|Score(Q,K)| = 1$.

$Score(Q,K$) ist also ein Vektor exakt der Länge von $V$, der angibt, mit welchem Anteil jeder Eintrag von $V$ in den Ausgabevektor $Attention(Q,K,V)$ eingehen soll. Dabei summiert sich $Score(Q,K)$ zu 1, ist also tatsächlich eine Gewichtung der Einträge von $V$.

Da mit $Score(Q,K)$ nun also eine Gewichtung besteht, wie stark die Ausgabe $Attention(Q,K,V)_i$ von $V_j$ abhängt, kann man dieses Verhältnis als Matrix grafisch darstellen. Dies geschieht in der Simulation unten, bei der für den vorgegebenen Satz eine Attentionsgewichtung aus einer der Attentionsschichten dargestellt wird. Der Eintrag in der $i-ten$ Zeile und $j-ten$ Spalte gibt dabei den einfluss von $V_j$ auf $Attention(Q,K,V)_i$ an.


In [ ]:
tokenizer_attn = StoryTokenizer(reserved_tokens, vocab_path)
attn_model = WordComplete(StoryTokenizer(reserved_tokens, vocab_path), model.model, max_length=32)

input_widget_attn = widgets.Text(
    value='A longer test sentence is more interesting, as it allows to see dependencies more clearly.',
    description='Ihre Eingabe:',
    continuous_update=False,  # updates value only when you finish typing or hit "Enter"
    layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
)

button_widget_attn = widgets.Button(description='Attention berechnen',
                               layout = widgets.Layout(width='auto'))

output_widget_attn = widgets.Output(layout = widgets.Layout(width='auto'))

def create_tokenized_embeddings():
        tensor_input = tf.convert_to_tensor(input_widget_attn.value)                # Umwandelung des Textinputs in ein TensorFlow-Tensor 
        output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)    # Erstellung eines leeren TensorArrays für die spätere Ausgabe
        if len(tensor_input.shape) == 0:                                            # Überprüft, ob der Eingabetensor im korrekten Format ist                                     
            tensor_input = tensor_input[tf.newaxis]                                 # Falls nicht, wird eine Dimension hinzufügt 

    
        tokenized_input = tokenizer.tokenize(tensor_input).to_tensor()              # Umwandlung des Textinputs in Tokens und anschließend in einen Tensor
        input_without_eos = tokenized_input[:, :-1]
        context = transformer.encode(input_without_eos, None)                       # Erstellung der Kontext-Vektoren vom Transformer-Modell

        # Write the input tokens (excluding the last one) to the output array
        for i, value in enumerate(tokenized_input[0][:-1]):
            output_array = output_array.write(i, value)

        dec_input = output_array.concat()[tf.newaxis]

        dec_out = transformer.decode(context, None, dec_input, None)

def on_button_click(b):
    with output_widget_attn:
        #VisualWrapper.n_vis_layers_per_class['MultiHeadedAttention'] = 6
        #output_widget_attn.clear_output()  # clear the previous output
        create_tokenized_embeddings()
        VisualWrapper.visualize_data(id='attention')
        #VisualWrapper.n_vis_layers_per_class['MultiHeadedAttention'] = 1
            

button_widget_attn.on_click(on_button_click)

display(input_widget_attn, button_widget_attn, output_widget_attn)

#### Multi-headed Attention

Multi-headed Attention ist eine Erweiterung des Attention-Mechanismus, die darauf abzielt, die komplexen Strukturen von Texten besser zu erfassen. In herkömmlichen Modellen konkurrieren zahlreiche Beziehungen und Verbindungen innerhalb eines Textes um die Aufmerksamkeit eines einzigen Mechanismus, was zu einer Überlastung führen kann. Durch die Einführung von Multi-headed Attention wird diese Einschränkung überwunden, indem mehrere, parallel arbeitende Attention-Ströme geschaffen werden, von denen sich jeder auf unterschiedliche Aspekte des Textes konzentriert.

Diese spezialisierten "Köpfe" können verschiedene Typen von Zusammenhängen innerhalb der Eingabedaten verarbeiten. Ein Kopf könnte sich auf die Beziehung zwischen Subjekten und Prädikaten konzentrieren, ein anderer auf die Kohärenz thematischer Elemente, und ein dritter könnte die Verbindung zwischen Haupt- und Nebensätzen analysieren. Durch diese Aufteilung wird vermieden, dass die "Köpfe" in Konkurrenz zueinander treten; stattdessen ergänzen sie sich, was zu einer umfassenderen Analyse führt.

Die resultierenden, von jedem Kopf generierten Outputs werden anschließend zu einer einzigen, zusammenhängenden Darstellung kombiniert. Diese Synthese bietet eine reichhaltige, vielschichtige Perspektive auf die Eingabedaten, die weit über das hinausgeht, was mit einem einzigen Attention-Mechanismus möglich wäre. Multi-headed Attention ist somit ein Schlüsselelement, das die Fähigkeit von Modellen verbessert, subtile und komplexe Muster in Daten zu erkennen und darauf zu reagieren.\

Mathematisch betrachtet werden dazu zu Beginn h gewichteten Matrizen 

$$W_i^Q, W_i^K, W_i^V i= 1,...,h$$ 

eingeführt. Diese erzeugen also h verschiedene Matrixtripel 

$$Q_i, K_i, V_i i = 1,...,h$$

und somit ergeben sich h verschiedene Ausgaben 

$$H_i = Attention(Q_i, K_i, V_i) i=1,...,h.$$

Diese werden nun zu einer einzigen Ausgabe zusammengeführt, indem wir 

$$MultiHeadAttention(Q_in,K_in,V_in) = Concat(H_1,..., H_h)W^O$$

berechnen. Dabei ist $Concat(A_1,...,A_n)$ das hintereinanderschreiben mehrerer Matrizen und $W^O$ eine weitere trainierbare Matrix, die die verschiedenen Ausgaben $H_1,..., H,h$ gewichtet.
Deshalb sehen wir in der obigen Ausgabe auch x verschiedene Matrizen, die $Score(Q_i,K_i)$ darstellen.




<figure id="fig:fig_masking" style="text-align: center; height: 450px;">
  <img src="./img/tf_mhattention.png" alt="Abbildung x: Beispiel Multi headed Attention."/>
  <figcaption>Abbildung 5: Beispiel Multi-headed Attention</figcaption>
</figure>
<br>

### Erklärung der Grafik

Die Abbildung zeigt zwei Beispiele für die Visualisierung von Multi-headed Attention in einem Transformer-Modell. Jedes Diagramm repräsentiert die Aufmerksamkeitsverteilung eines eigenen "Kopfes" innerhalb des Attention-Mechanismus, und zwar für einen gegebenen Satz "Ich besuchte den Kurs Digital Leadership und lernte".
In beiden Diagrammen sind die vertikalen Balken proportional zur Stärke der Aufmerksamkeit, die jedes Wort vom jeweiligen Kopf erhält. Ein höherer Balken bedeutet, dass das entsprechende Wort eine stärkere Aufmerksamkeit erhält, wenn das Modell versucht, die Bedeutung des Satzes zu interpretieren oder eine Aufgabe wie die Übersetzung durchzuführen.

Die Wörter am unteren Rand jedes Diagramms sind die Eingabewörter, und die kleinen "v" und "k" Symbole repräsentieren die Values und Keys im Attention-Mechanismus. Das "q" steht für den Query-Vektor, der in diesem Fall auf das Wort "lernte" zeigt, was bedeutet, dass die Visualisierung die Aufmerksamkeit aus der Perspektive dieses spezifischen Wortes darstellt.

Attention-Kopf 1 fokussiert sich auf die Entitäten. Hier sehen wir, dass dieser Kopf vor allem auf die Wörter "Kurs", "Digital", und "Leadership" Aufmerksamkeit legt. Diese Wörter sind als Entitäten (Namen von Personen, Orten oder spezifischen Objekten) identifiziert worden, was darauf hindeutet, dass dieser Kopf darauf trainiert ist, solche Entitäten im Text zu erkennen und hervorzuheben.
Rechts: Attention-Kopf 2 fokussiert sich auf die syntaktisch relevanten Wörter

Der zweite Kopf scheint die Aufmerksamkeit auf die Wörter "Ich", "besuchte" aber auch das query Wort "lernte" selbst zu richten. Die beiden letzten Wörter sind Verben und "Ich" ist das zugehörige Pronomen. Dieser Kopf ist somit auf die Identifizierung syntaktischer Strukturen ausgerichtet.



### Masking

Das Maskieren des Inputs ist eine wichtige  Komponente der Transformerarchitektur. Beim Masking handelt es sich in Wirklichkeit um zwei Mechanismen, die zwar dieselbe Funktionsweise besitzen, aber sehr unterschiedliche Aufgaben in der Architektur übernehmen. Einerseits das Subsequent Masking, andererseits das Padding Masking. Das Padding Masking stellt jediglich sicher, dass nur Positionen mit Inhalt vom Transformer verarbeitet werden, während das Subsequent Masking dafür sorgt, dass der Decoder des Transformers autoregressiv ist. Das bedeutet, bei einer Vorhersage für eine Ausgabe an der Position $i$, soll das Modell nur Informationen aus den Eingabepositionen $1,…,i-1$ nutzen. Das Zusammenspiel der Beiden sehen Sie in der folgenden Abbildung (<a href="#fig:fig_masking">Abbildung 6</a>)

#### Padding Masking

Das Padding Masking ist notwendig, da Transformer sequenzielle Daten so verarbeiten, als ob sie eine fixe Länge $d_{model}$ hätten. Das geschieht deshalb, weil Transformer so lernen können jede Position der Ausgabelänge $d_{model}$ parallel vorherzusagen. 

Um während des Trainings auch Daten mit einer Länge größer oder kleiner $d_{model}$ zu nutzen, werden längere Sequenzen abgeschnitten und kürzere mit Nullen aufgefüllt. Diese Nullen müssen dann mit Hilfe von Padding Masking für das Training irrelevant gemacht werden. Das geschieht indem man alle Positionen die eine Null enthalten für das Modell auf $-inf$ setzt, sodass sie beim Gradient Descent nicht berücksichtigt werden.

#### Subsequent Masking

Das Subsequent Masking benutzt die gleich Technik des Werte auf $-inf$ setzen. Dabei wird aber nicht das mit irrelevanten Informationen angefüllte Ende des Eingabevektors auf Null gesetzt, sondern es werden diejenige Werte von $Score(Q, V)$ auf $-inf$ gesetzt, die einen Wert $V_j$, für die Ausgabe $i$ $Attention(Q,K,V)_i$ gewichten würden, obwohl $j>i$ gilt. Also es wird beschränkt, dass $Score(Q, V)$ nur diejenigen Werte von $V$ einbeziehen darf, die bei der Vorhersage für den i-ten Wert schon bekannt sind.

Dass das relevant ist liegt daran, dass beim Training von Transformern der komplette Input gleichzeitig verwertet wird. Ein Satz wird als Ganzes vom Transformer verarbeitet und er erzeugt eine Vorhersage, welches Token an einer bestimmten Position in diesem Satz vorkommt. Dabei erhält er in der Eingabe aber schon die Information, welches Token an dieser Position tatsächlich steht, da er den kompletten Satz als Eingabe bekommen hat. Dies muss nun also innerhalb des Modells mit einer Maskierung wieder rückgängig gemacht.

<figure id="fig:fig_masking" style="text-align: center; height: 700px;">
  <img src="./img/tf_masking.jpg" alt="Abbildung 6: Zwei Varianten des Maskings im Transformer Modell."/>
  <figcaption>Abbildung 6: Die zwei Varianten des Maskings im Transformer Modell</figcaption>
</figure>
<br>


### Verschiedene Attention-Mechanismen

In der Architektur werden verschiedene Attentionstypen unterschieden. Es gibt dabei zwei Variablen die beeinflussen, welche Art von Attention wir verwenden. Die erste Variable ist, woher die Eingaben $Q', K' und V'$ kommen. Die zweite Variable ist die Maskierung, die wir auf $Score(Q,K)$ anwenden.

#### Self-Attention

Wir sprechen von Self-Attention, wenn gilt 

$$Q'=K'=V'.$$ 

Wenn sich $Score(Q,K)$ also bildlich gesprochen daraus ergibt, welche Attention jede Position der Eingabe auf eine andere Position derselben Eingabe legt und diese Attention auf die Eingabe selbst angewandt wird.

#### Cross-Attention

Wie sprechen von Cross-Attention, wenn gilt 

$$K' = V'$$

aber $Q'$ von diesen beiden Werten verschieden ist.
Wenn sich $Score(Q,K)$ also daraus ergibt, welche Attention jede Position einer Eingabe $Q'$ auf die Positionen einer zweiten Eingabe $K'$ gibt und dieser Attentionsscore auf die zweite Eingabe angewandt wird.

Dies ist zum Beispiel in Transformern der Fall, wenn $Q'$ sich aus der Ausgabe des Encoder ergibt und $K' = V'$ ein Zwischenergebnis des Decoders ist.

#### Masked Attention

Ein Fall von Masked-Attention liegt dann vor, wenn bestimmte Werte von $Score(Q,K)$ maskiert werden. Das ist zum Beispiel beim Subsequent Masking der Fall, hier wird $Score(Q,K)i,j = -inf$ gesetzt für alle Einträge $j>i$. Dadurch wird verhindert, dass die Ausgabe $Attention(Q,K,V)_i$ sich auf die Werte $V_j, j>i $ stützt. Zum Beispiel wird während des Trainings im Decoder dadurch verhindert, dass das Model lernt Informationen aus den zukünftigen Einträgen $V_j, j>i$ zu benutzen, um $V_i$ vorherzusagen. Man sieht gut in der Darstellung von $Score(Q,K)$, dass die Werte für $j>i$ dadurch meistens nahe bei $0$ liegen.


### Skalierung mit sqr(d_k)

Dasselbe Problem des Vanishing Gradient könnte innerhalb der Attention-Funktion auftauchen, da hier $softmax(QK^T)V$ berechnet wird, das Skalarprodukt $QK^T$ mit $d_{model}$ skaliert und die Softmaxfunktion
Das Problem des Vanishing Gradients kann auch innerhalb der Attention Function auftreten, da hier softmax(QK^T)V berechnet wird und das Skalarprodukt QK^T mit *d_model* skaliert und die Softmaxfunktion
    
$$sigma(z)_i = exp(z_i)/sum_j=1^N(exp(z_j)) für j=1,...,N $$

somit leicht in einen saturierten Bereich mit extrem kleinen Gradienten gerät. Deshalb wird in der Umsetzung der Architektur $QK^T$ mit $sqr(d_{model})$ skaliert: 
$$softmax(QK^T/sqr(d_{model}))$$, 
um die Skalierung mit $d_{model}$ zu minimieren.

## Vollständige Simulation

Zuletzt findet sich hier nun noch eine Simulation des kompletten Inferenzvorgangs innerhalb eines Transformermodells. In dieser Simulation kann man noch einmal alle vorher beschriebenen Schritte nachvollziehen.

In [ ]:
inference_model = WordComplete(StoryTokenizer(reserved_tokens, vocab_path), model.model, max_length=32)

input_widget_inf = widgets.Text(
    value='Test sentence',
    description='Your input:',
    continuous_update=False,  
    layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
)

button_widget_inf = widgets.Button(description='Run interactive inference',
                               layout = widgets.Layout(width='auto'))

output_widget_inf = widgets.Output(layout = widgets.Layout(width='auto'))

def on_button_click(b):
    with output_widget_inf:
        output_widget_inf.clear_output()  
        inference_model(input_widget_inf.value, interactive=True) 
        inference_model.print_results(visualisation=True)

button_widget_inf.on_click(on_button_click)

display(input_widget_inf, button_widget_inf, output_widget_inf)


# Bibliography
[1] Vaswani, A. et al. Attention Is All You Need. Preprint at http://arxiv.org/abs/1706.03762 (2017).

[2] Hochreiter, S. & Schmidhuber, J. Long Short-Term Memory. Neural Computation 9, 1735–1780 (1997).

[3] Bengio, Y., Simard, P. & Frasconi, P. Learning long-term dependencies with gradient descent is difficult. IEEE Transactions on Neural Networks 5, 157–166 (1994).

[4] Cho, K., van Merrienboer, B., Bahdanau, D. & Bengio, Y. On the Properties of Neural Machine Translation: Encoder–Decoder Approaches. in Proceedings of SSST-8, Eighth Workshop on Syntax, Semantics and Structure in Statistical Translation 103–111 (Association for Computational Linguistics, 2014). doi:10.3115/v1/W14-4012.

[6] Kaplan, J. et al. Scaling Laws for Neural Language Models. Preprint at http://arxiv.org/abs/2001.08361 (2020).

[5] Goodfellow, I., Bengio, Y. & Courville, A. Deep learning. (The MIT Press, 2016).

[7] Radford, A. et al. Language Models are Unsupervised Multitask Learners. (2019).

[8] Devlin, J., Chang, M.-W., Lee, K. & Toutanova, K. BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. Preprint at http://arxiv.org/abs/1810.04805 (2019).

[9] Alammar, J. The Illustrated Transformer – Jay Alammar – Visualizing machine learning one concept at a time. https://jalammar.github.io/illustrated-transformer/ (2018).

[10] Encoder-Decoder. Understanding The Model Architecture | by Naoki | Medium. https://naokishibuya.medium.com/transformers-encoder-decoder-434603d19e1.

[11] Srivastava, N., Hinton, G., Krizhevsky, A., Sutskever, I. & Salakhutdinov, R. Dropout: A Simple Way to Prevent Neural Networks from Overﬁtting. (2014).

[12] He, K., Zhang, X., Ren, S. & Sun, J. Deep Residual Learning for Image Recognition. Preprint at http://arxiv.org/abs/1512.03385 (2015).

[13] Gage, P. A New Algorithm for Data Compression. (1994).

[14] Cho, K. et al. Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation. Preprint at http://arxiv.org/abs/1406.1078 (2014).

[15] Bahdanau, D., Cho, K. & Bengio, Y. Neural Machine Translation by Jointly Learning to Align and Translate. Preprint at http://arxiv.org/abs/1409.0473 (2016).

[16] OpenAI. GPT-4 Technical Report. Preprint at http://arxiv.org/abs/2303.08774 (2023).

[17] Grefenstette, G. & Tapanainen, P. What is a word, What is a sentence? Problems of Tokenization.

[18] Lin, Z. et al. A Structured Self-attentive Sentence Embedding. Preprint at http://arxiv.org/abs/1703.03130 (2017).

[19] Schmidt, R. M. Recurrent Neural Networks (RNNs): A gentle Introduction and Overview. Preprint at http://arxiv.org/abs/1912.05911 (2019).

[20] Ioffe, S. & Szegedy, C. Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift. Preprint at http://arxiv.org/abs/1502.03167 (2015).

[21] Ba, J. L., Kiros, J. R. & Hinton, G. E. Layer Normalization. Preprint at http://arxiv.org/abs/1607.06450 (2016).

[22] He, K., Zhang, X., Ren, S. & Sun, J. Deep Residual Learning for Image Recognition. Preprint at http://arxiv.org/abs/1512.03385 (2015).

[23] Tunstall, L., von Werra, L., Wolf, T. Natural Language Processing Mit Transformern. https://www.oreilly.com/library/view/natural-language-processing/9781098157081/.
